In [1]:
_PROMPT_TEMPLATE = """
用户会提出一个需要你回答的问题，你应该对问题进行理解回答。

对于用户的问题，你输出的内容应该是一个一行的字符串，下面是一个例子

例子:
用户：蒙古国的首都是
回答：蒙古国的首都是乌兰巴托（Ulaanbaatar）


你的回答格式应该按照下面的内容，请注意“---output”等标记都必须输出，这是我用来提取答案的标记。
不要输出中文的逗号，不要输出引号。除了格式示例的内容，你不应该输出其他文字。

格式示例：
---input
用户的问题
---output
你的答案


现在，我们开始作答
问题: 中国的首都是？
"""

In [1]:
import dashscope
from dashscope import Generation
from http import HTTPStatus
import random
from CustomLLM import QWENonlie
apikey='sk-02b8b1a9175a4cae9bf9571d517d8b81'
llm=QWENonlie.from_name(model_name=Generation.Models.qwen_max,api_key=apikey)

In [2]:
from config import QWEN_CONFIG
from Container import g_container
from dashscope import TextEmbedding,BatchTextEmbedding
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader
from langchain.embeddings import DashScopeEmbeddings
from TxtProcess import faiss_kb_ds
g_container.MODEL = llm
cfg = QWEN_CONFIG(api_key=apikey,
                  embedding_name=BatchTextEmbedding.Models.text_embedding_async_v1)
emb=DashScopeEmbeddings(model=TextEmbedding.Models.text_embedding_v1,
                        dashscope_api_key=apikey)
text_ld=TextLoader(file_path='./test_data/03c625c108ac0137f413dfd4136adb55c74b3805.txt')
sp=RecursiveCharacterTextSplitter(
        chunk_size=cfg.chunk_size, chunk_overlap=cfg.chunk_overlap)
docs=text_ld.load_and_split(text_splitter=sp)
fkb=faiss_kb_ds(vs_path=cfg.vec_store_path,kb_path='./test_data/',embs=emb)
fkb.do_add_doc(docs)

[{'id': '5b9df0d1-8bd8-456d-b0c2-4e726ee7ef5f',
  'meta_data': {'source': './test_data/03c625c108ac0137f413dfd4136adb55c74b3805.txt'}},
 {'id': '106ad076-ecf6-4437-9f88-41f3521cb496',
  'meta_data': {'source': './test_data/03c625c108ac0137f413dfd4136adb55c74b3805.txt'}},
 {'id': '9d60f1fb-2fec-4d78-80c1-d686951c8d1a',
  'meta_data': {'source': './test_data/03c625c108ac0137f413dfd4136adb55c74b3805.txt'}},
 {'id': 'ccb9fa4a-2deb-431f-839c-3dcb61ca249d',
  'meta_data': {'source': './test_data/03c625c108ac0137f413dfd4136adb55c74b3805.txt'}},
 {'id': 'ae35ed64-00f5-4130-a096-fe54cc772109',
  'meta_data': {'source': './test_data/03c625c108ac0137f413dfd4136adb55c74b3805.txt'}},
 {'id': 'ba02ddac-a47f-4b3d-87b3-faf50c29cce4',
  'meta_data': {'source': './test_data/03c625c108ac0137f413dfd4136adb55c74b3805.txt'}},
 {'id': 'd7541cf1-cc87-4d17-af00-508daa38f9fc',
  'meta_data': {'source': './test_data/03c625c108ac0137f413dfd4136adb55c74b3805.txt'}},
 {'id': '82e29303-6232-49f5-8d28-b9b5dbd09055',


In [6]:
fkb.do_search(query='固定资产',top_k=5,threshold=1)

[(Document(page_content='报告期期末，公司固定资产账面净值分别为19,161.80万元、10,470.75万\n元、6,176.69万元和6,666.66万元，占各期非流动资产总额比重分别为76.31%、\n43.13%、29.84%和39.91%。\n截至2016年6月30日，公司的固定资产构成及累计折旧情况如下：\n<|TABLE_0280_0002.xlsx|>\n<|TABLE_0280_0003.xlsx|>\n<|TABLE_0281_0000.xlsx|>\nB.2015年度固定资产变动情况如下:\n<|TABLE_0281_0001.xlsx|>\n<|TABLE_0282_0000.xlsx|>\n<|TABLE_0282_0001.xlsx|>\n<|TABLE_0283_0000.xlsx|>\n<|TABLE_0283_0001.xlsx|>\n<|TABLE_0284_0000.xlsx|>\n②2013年-2015年，发行人拥有48条生产线；2016年5月底，新增明胶空\n心胶囊生产线16条。\n③报告期内，机器设备的变动与产能变化情况如下:\n<|TABLE_0284_0001.xlsx|>', metadata={'source': './test_data/03c625c108ac0137f413dfd4136adb55c74b3805.txt'}),
  0.6905935),
 (Document(page_content='额之间的差额，计入丧失控制权当期的投资收益，同时冲减商誉。与原有子公\n司股权投资相关的其他综合收益等，应当在丧失控制权时转为当期投资收益。\n2）通过多次交易分步处置对子公司投资至丧失控制权，且属于“一揽子交\n易”的\n将各项交易作为一项处置子公司并丧失控制权的交易进行会计处理。但是，\n在丧失控制权之前每一次处置价款与处置投资对应的享有该子公司净资产份额\n的差额，在合并财务报表中确认为其他综合收益，在丧失控制权时一并转入丧\n失控制权当期的损益。\n（十）投资性房地产\n1、投资性房地产包括已出租的土地使用权、持有并准备增值后转让的土地\n使用权和已出租的建筑物。\n2、投资性房地产按照成本进行初始计量，采用成本模式进行后续计量，并\n采用与固定资产和无形资产相同的方法计提折